# Introduction

The objective of this project is to investigate recent trends in NYC 311 call data and develop a neural network capable of categorizing the responding agency for each call. By gaining insights into the content, timing, frequency, and geographical distribution of 311 call issues, government agencies and local policymakers can better address the needs of NYC residents. This project aims to enhance this understanding through exploratory data analysis and visualizations of public 311 data. Additionally, the modeling phase, which focuses on agency classification, aims to demonstrate the feasibility of a tool that can seamlessly connect residents with the right government services.

# Data Sources

The data used in this project was obtained from these sources:

NYC Open Data's 311 Service Requests from 2010 to Present: 
This dataset contains information about the time, location, complaint type, and status of more than 24 million 311 service requests made in New York City within the past decade. This project uses a subset of the data from 2020 that was accessed with the Socrata Open Data (SODA) API. (https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9)

# Obtaining the 311 Data

The API Documentation for this dataset contains further information about how to obtain filtered versions of the data. Below, we simply request the 1500000 most recent 311 calls. The line client.timeout = 1000 prevents the Socrata connection from timing out after its default setting of 10 seconds.

In [1]:
import pandas as pd
from sodapy import Socrata

client = Socrata("data.cityofnewyork.us", None)
client.timeout = 1000
results = client.get("erm2-nwe9", limit=1500000)

# Convert to pandas DataFrame
df = pd.DataFrame.from_records(results)

# Preprocessing Data

## Dealing with Missing and Superfluous Data
Let's check the column data types and identify columns with missing values.

In [2]:
df.shape

(1500000, 47)

In [3]:
#check columns and it's datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500000 entries, 0 to 1499999
Data columns (total 47 columns):
 #   Column                          Non-Null Count    Dtype 
---  ------                          --------------    ----- 
 0   unique_key                      1500000 non-null  object
 1   created_date                    1500000 non-null  object
 2   agency                          1500000 non-null  object
 3   agency_name                     1500000 non-null  object
 4   complaint_type                  1500000 non-null  object
 5   descriptor                      1477839 non-null  object
 6   location_type                   1328443 non-null  object
 7   incident_zip                    1479825 non-null  object
 8   incident_address                1445100 non-null  object
 9   street_name                     1445066 non-null  object
 10  cross_street_1                  1071059 non-null  object
 11  cross_street_2                  1071512 non-null  object
 12  address_type  

In [4]:
#check for null values
df.isnull().sum()

unique_key                              0
created_date                            0
agency                                  0
agency_name                             0
complaint_type                          0
descriptor                          22161
location_type                      171557
incident_zip                        20175
incident_address                    54900
street_name                         54934
cross_street_1                     428941
cross_street_2                     428488
address_type                         8634
city                                80578
status                                  0
resolution_description              67573
resolution_action_updated_date      48946
community_board                         0
bbl                                174146
borough                                 0
x_coordinate_state_plane            20663
y_coordinate_state_plane            20107
open_data_channel_type                  0
park_facility_name                

In [5]:
#eliminate rows without lat/long and location type data
df = df[df.latitude.notnull()]
df = df[df.longitude.notnull()]
df = df[df.descriptor.notnull()]

In [6]:
#Including only necessary columns
df = df[['unique_key','created_date', 'complaint_type', 'agency', 'community_board', 
         'latitude', 'longitude', 'descriptor', 'closed_date','city','borough',
         'incident_zip','incident_address' ,'status','street_name','address_type','resolution_description']]

### Descriptions of Remaining Columns
The descriptions for every column are available on the dataset's NYCOpenData profile. The descriptions for the columns included in this analysis have been pasted below.
* Unique Key: Unique identifier of a Service Request (SR) in the open data set
* Created Date: Date service request(SR) was created
* Complaint Type: This is the first level of a hierarchy identifying the topic of the incident or condition. Complaint Type may have a corresponding Descriptor (below) or may stand alone.
* Agency: Acronym of responding City Government Agency
* Community Board: Provided by geovalidation.
* Latitude: Geo based Lat of the incident location
* Longitude: eo based Long of the incident location
* Descriptor: This is associated to the Complaint Type, and provides further detail on the incident or condition. Descriptor values are dependent on the Complaint Type, and are not always required in SR.
* Borough: Provided by the submitter and confirmed by geovalidation.
* Closed Date: Date SR was closed by responding agency
* City: City of the incident location provided by geovalidation.
* Incident Zip: Incident location zip code, provided by geo validation.
* Incident Address: House number of incident address provided by submitter.
* Status: Status of SR submitted
* Street Name: Street name of incident address provided by the submitter
* Address Type: Type of incident location information available.
* Resolution Description: Describes the last action taken on the SR by the responding agency. May describe next or future steps.

In [7]:
#convert date column to datetime 
df['created_date'] = pd.to_datetime(df['created_date'], 
                                    format='%Y-%m-%dT%H:%M:%S.%f')

In [8]:
#convert date column to datetime 
df['closed_date'] = pd.to_datetime(df['closed_date'], 
                                    format='%Y-%m-%dT%H:%M:%S.%f')

Using the datetime format, we can now create new columns with the day, month, day of week, and hour of each call. This will allow us to explore the independent relationship of each variable to other characteristics of the calls.

In [9]:
df['day'] = [str(i.date()) for i in df.created_date]
df['month'] = [int(i.month) for i in df.created_date]
df['day_of_week'] = [int(i.weekday()) for i in df.created_date]
df['hour'] = [int(i.hour) for i in df.created_date]
df['year'] = [int(i.year) for i in df.created_date]

In [10]:
df.columns

Index(['unique_key', 'created_date', 'complaint_type', 'agency',
       'community_board', 'latitude', 'longitude', 'descriptor', 'closed_date',
       'city', 'borough', 'incident_zip', 'incident_address', 'status',
       'street_name', 'address_type', 'resolution_description', 'day', 'month',
       'day_of_week', 'hour', 'year'],
      dtype='object')

In [11]:
df.describe()

,month,day_of_week,hour,year
count,1.457258e+06,1.457258e+06,1.457258e+06,1457258.0
mean,9.402193e+00,2.859345e+00,1.307019e+01,2023.0
std,1.548507e+00,1.970384e+00,6.096539e+00,0.0
min,7.000000e+00,0.000000e+00,0.000000e+00,2023.0
25%,8.000000e+00,1.000000e+00,9.000000e+00,2023.0
50%,9.000000e+00,3.000000e+00,1.300000e+01,2023.0
75%,1.100000e+01,4.000000e+00,1.800000e+01,2023.0
max,1.200000e+01,6.000000e+00,2.300000e+01,2023.0


In [12]:
#to Categorize the incident time of the day lets create a function 
time_slots = {
    'Earlymorning': (0, 6),
    'Morning': (7, 11),
    'Afternoon': (12, 16),
    'Evening': (17, 21),
    'Night': (22, 23)
}

# Function to categorize timestamps into slots
def categorize_timestamp(timestamp):
    hour = timestamp.hour
    for slot, (start_hour, end_hour) in time_slots.items():
        if start_hour <= hour <= end_hour:
            return slot
    return 'unknown'  # If the timestamp doesn't fit into any slot


In [13]:
#apply categorize_timestamp on created date to get time_of_day
df['time_of_day'] = df['created_date'].apply(categorize_timestamp)

In [14]:
#check the column
df.head()

,unique_key,created_date,complaint_type,agency,community_board,latitude,longitude,descriptor,closed_date,city,...,status,street_name,address_type,resolution_description,day,month,day_of_week,hour,year,time_of_day
0,59753376,2023-12-16 12:00:00,Derelict Vehicles,DSNY,07 BRONX,40.87258237734268,-73.87671883675628,Derelict Vehicles,NaT,BRONX,...,Open,DECATUR AVENUE,ADDRESS,If the abandoned vehicle meets the criteria to...,2023-12-16,12,5,12,2023,Afternoon
1,59753375,2023-12-16 12:00:00,Derelict Vehicles,DSNY,10 BRONX,40.84077130162347,-73.85115751944943,Derelict Vehicles,NaT,BRONX,...,Open,BUCK STREET,ADDRESS,If the abandoned vehicle meets the criteria to...,2023-12-16,12,5,12,2023,Afternoon
2,59754726,2023-12-16 12:00:00,Derelict Vehicles,DSNY,10 BRONX,40.84077130162347,-73.85115751944943,Derelict Vehicles,NaT,BRONX,...,Open,BUCK STREET,ADDRESS,If the abandoned vehicle meets the criteria to...,2023-12-16,12,5,12,2023,Afternoon
3,59753377,2023-12-16 12:00:00,Derelict Vehicles,DSNY,12 QUEENS,40.687698072630916,-73.77855037697357,Derelict Vehicles,NaT,JAMAICA,...,Open,116 AVENUE,ADDRESS,If the abandoned vehicle meets the criteria to...,2023-12-16,12,5,12,2023,Afternoon
4,59749200,2023-12-16 12:00:00,Derelict Vehicles,DSNY,10 MANHATTAN,40.825981342318265,-73.94089757369808,Derelict Vehicles,NaT,NEW YORK,...,Open,BRADHURST AVENUE,ADDRESS,If the abandoned vehicle meets the criteria to...,2023-12-16,12,5,12,2023,Afternoon


In [15]:
#ckeck for null values
df.isnull().sum()

unique_key                     0
created_date                   0
complaint_type                 0
agency                         0
community_board                0
latitude                       0
longitude                      0
descriptor                     0
closed_date               126238
city                       59483
borough                        0
incident_zip                5308
incident_address           41172
status                         0
street_name                41201
address_type                5200
resolution_description     66714
day                            0
month                          0
day_of_week                    0
hour                           0
year                           0
time_of_day                    0
dtype: int64

In [16]:
#drop rows with null values
df = df.dropna()

In [17]:
df.isnull().sum()

unique_key                0
created_date              0
complaint_type            0
agency                    0
community_board           0
latitude                  0
longitude                 0
descriptor                0
closed_date               0
city                      0
borough                   0
incident_zip              0
incident_address          0
status                    0
street_name               0
address_type              0
resolution_description    0
day                       0
month                     0
day_of_week               0
hour                      0
year                      0
time_of_day               0
dtype: int64

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1223546 entries, 46 to 1499999
Data columns (total 23 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   unique_key              1223546 non-null  object        
 1   created_date            1223546 non-null  datetime64[ns]
 2   complaint_type          1223546 non-null  object        
 3   agency                  1223546 non-null  object        
 4   community_board         1223546 non-null  object        
 5   latitude                1223546 non-null  object        
 6   longitude               1223546 non-null  object        
 7   descriptor              1223546 non-null  object        
 8   closed_date             1223546 non-null  datetime64[ns]
 9   city                    1223546 non-null  object        
 10  borough                 1223546 non-null  object        
 11  incident_zip            1223546 non-null  object        
 12  incident_addr

In [19]:
#chege datatype of columns 'unique_key','incident_zip' to integer 
columns_to_convert_to_int = ['unique_key', 'incident_zip']
df[columns_to_convert_to_int] = df[columns_to_convert_to_int].astype(int)

Calculate total hours took to solve the request

In [20]:
df['total_hours'] = (df['closed_date'] - df['created_date']).dt.total_seconds() / 3600

In [21]:
df['total_hours'].head(5)

46    0.100000
50    0.017222
63    0.013889
68    0.257500
70    0.196111
Name: total_hours, dtype: float64

The values in the 'agengy' column also have inconsistent formatting: all but one have abbreviated names.

In [22]:
df['agency'].unique()

array(['NYPD', 'DSNY', 'DHS', 'DEP', 'DPR', 'HPD', 'DOB', 'DOT', 'TLC',
       'DCWP', 'DOHMH', 'DOE', 'EDC'], dtype=object)

We'll convert the ''DEPARTMENT OF CONSUMER AND WORKER PROTECTION'value to its abbreviated name, DCWP:

In [23]:
def update_DCWP(agency):

  """Takes in an agency name and converts to abbreviated form, if the agency is
     the DEPARTMENT OF CONSUMER AND WORKER PROTECTION."""

  if agency == 'DEPARTMENT OF CONSUMER AND WORKER PROTECTION':
      return "DCWP"
  else:
      return agency
    
df.agency = df.agency.map(lambda agency: update_DCWP(agency))
     

In [24]:
#check if changes are reflected
df['agency'].unique()

array(['NYPD', 'DSNY', 'DHS', 'DEP', 'DPR', 'HPD', 'DOB', 'DOT', 'TLC',
       'DCWP', 'DOHMH', 'DOE', 'EDC'], dtype=object)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1223546 entries, 46 to 1499999
Data columns (total 24 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   unique_key              1223546 non-null  int64         
 1   created_date            1223546 non-null  datetime64[ns]
 2   complaint_type          1223546 non-null  object        
 3   agency                  1223546 non-null  object        
 4   community_board         1223546 non-null  object        
 5   latitude                1223546 non-null  object        
 6   longitude               1223546 non-null  object        
 7   descriptor              1223546 non-null  object        
 8   closed_date             1223546 non-null  datetime64[ns]
 9   city                    1223546 non-null  object        
 10  borough                 1223546 non-null  object        
 11  incident_zip            1223546 non-null  int64         
 12  incident_addr

In [26]:
df.shape

(1223546, 24)

In [27]:
#The (NYC 311) dataset contains complaints logged since what date?
df['created_date'].min()

Timestamp('2023-07-09 19:07:08')

In [28]:
#The (NYC 311) dataset contains complaints logged till what date?
df['created_date'].max()

Timestamp('2023-12-16 00:56:44')

## Reformatting Values for Consistency
If we look at unique values in the 'complaint type' column, we'll see that some values are in title case while others are in upper case.

In [29]:
df.complaint_type.unique()

array(['Noise - Residential', 'Noise - Commercial',
       'Noise - Street/Sidewalk', 'Illegal Parking', 'Blocked Driveway',
       'Noise - Vehicle', 'Encampment', 'Vendor Enforcement',
       'Abandoned Vehicle', 'Homeless Person Assistance', 'Traffic',
       'Graffiti', 'Non-Emergency Police Matter', 'Water System',
       'Drinking', 'Animal-Abuse', 'Animal in a Park',
       'Bike/Roller/Skate Chronic', 'Illegal Fireworks', 'Sewer',
       'Urinating in Public', 'HEAT/HOT WATER', 'Noise - Park',
       'Elevator', 'Drug Activity', 'General Construction/Plumbing',
       'Emergency Response Team (ERT)', 'Building/Use',
       'Special Projects Inspection Team (SPIT)', 'Real Time Enforcement',
       'Electrical', 'Plumbing', 'Noise - House of Worship',
       'BEST/Site Safety', 'Panhandling',
       'Investigations and Discipline (IAD)', 'Dirty Condition',
       'PLUMBING', 'Violation of Park Rules',
       'Residential Disposal Complaint', 'Obstruction', 'DOOR/WINDOW',
       '

Below, all of the values in the column are converted to title case.

In [30]:
df.complaint_type = df.complaint_type.str.title()

Among noise-related complaints, only one does not specify the noise source:

In [31]:
noise_columns = []

for i in df.complaint_type:
   if i.startswith('Noise') and i not in noise_columns:
     noise_columns.append(i)
noise_columns

['Noise - Residential',
 'Noise - Commercial',
 'Noise - Street/Sidewalk',
 'Noise - Vehicle',
 'Noise - Park',
 'Noise - House Of Worship',
 'Noise',
 'Noise - Helicopter']

We'll convert the generic 'Noise' value to 'Noise - Unspecified' so that its formatting matches that of similar complaint types.

In [32]:
#Label unspecified noise complaints as 'Noise - Unspecified'

def update_noise(complaint):

  """Takes in an complaint name and updates it, if it is an unspecified noise complaint."""

  if complaint == 'Noise':
    
    return "Noise - Unspecified"

  else:

    return complaint
    
df.complaint_type = df.complaint_type.map(
    lambda complaint: update_noise(complaint))

The calls are currently ordered in the dataframe from most-least recent. For data visualization purposes, we'll reindex the calls so that the linear, left-to-right presentation of time on an x-axis runs from least-most recent.

In [33]:
#reorder df from least-most recent calls
df = df.reindex(index=df.index[::-1])     

To numerically encode the descriptor column in the modeling phase, we'll first need to exclude all values that aren't letters. Below, an empty space is substituted for all numbers and non-letter symbols.

In [34]:
import re
num_pattern = r'[-()0-9]'
df.descriptor = df.descriptor.map(lambda x: re.sub(num_pattern, '', x).lower())
df.descriptor = df.descriptor.map(lambda x: re.sub('/', ' ', x))

In [35]:
#preview sample of unique descriptors with non-letters removed
df.descriptor.unique()[0:20]

array(['hitting phone cable lines', 'sign', 'banging pounding',
       'construction debris', 'blocked hydrant',
       'catch basin clogged flooding use comments sc',
       'sewer backup use comments sa', 'chronic dumping',
       'posted parking sign violation', 'loud music party', 'trash',
       'loud talking', 'with license plate',
       'double parked blocking traffic',
       'noise: construction before after hours nm', 'neglected',
       'double parked blocking vehicle', 'engine idling',
       'paper license plates', 'cavein'], dtype=object)

In [36]:
# convert a numeric month column to a text month column 
month_names = [
        'January', 'February', 'March', 'April',
        'May', 'June', 'July', 'August',
        'September', 'October', 'November', 'December'
    ]
df['month'] = df['month'].apply(lambda x: month_names[x - 1] if 1 <= x <= 12 else 'Invalid Month')

# Exporting the Data

In [37]:
df.to_csv('nyc_311_initial_new.csv', header=True)